### Face Matching only

In [4]:
import cv2
import numpy as np
import face_recognition
import os

In [5]:
path = r'C:\Users\JOKER\Documents\Face_exp\CSE-1A PHOTOS'  # here collected dataset that we used for recognition
images = []                                              # Only for images
classNames = []  # only for images names  note: image withrespective name of images
myList = os.listdir(path)  #taking the path access
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')     #accessing the all images 
    images.append(curImg)                     # storing the all images
    classNames.append(os.path.splitext(cl)[0])    # storing the names of the images 

encodeListKnown=np.load('encodeface.npy')    # loading the encoding file of the collected images it will store in array formate

In [11]:
def findEncodings(images):       #this block is used for the training the images in the digital format 
    encodeList = []              # empty list for storing the encoded images for recognition
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)            # converting images into original format 
        encode = face_recognition.face_encodings(img)[0]        # storing the encoded images in encode
        encodeList.append(encode)                               # storing the all encoded images
    return encodeList
encodeListKnown = findEncodings(images)                      # return the encoded data 
#np.save('encodeface.npy',encodeListKnown)   # here encoded data takes time so for reduce the time we save the encode data

In [6]:
cap=cv2.VideoCapture(0)    # web cam or camara accessing command
while True:
    ret,frame= cap.read()                            # here frame is the photo for every secound
    imgS = cv2.resize(frame, (0, 0), None, 0.25, 0.25)   # resize the frame may notbe neccessery
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)        # convert the color
    facesCurFrame = face_recognition.face_locations(imgS)        # loacting the face in the frame
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)  # encoded the faces in the frame
    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
            matches = face_recognition.compare_faces(encodeListKnown, encodeFace)   # compare the encoded face from cam and encoded faces from colleceted data
            faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)  # find the which position of the faces is matched such if face is matched at six face then return the 6 values in array format
            matchIndex = np.argmin(faceDis)  # convert the array to normal number such as integer format
    
            if matches[matchIndex]:     
                name = classNames[matchIndex].upper()   # finding the name via using the matchIndex number
                y1, x2, y2, x1 = faceLoc        # loacation of the faces which are matched in x/4 ratio format
                y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4       # converting the ratio in original format
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)    # creating the rectangle on matched faces
                cv2.rectangle(frame, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)  # this is used for putting the recognized name
                cv2.putText(frame, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)  # name text

    cv2.imshow('testing faces',frame)   # return the frame after recognition
    if cv2.waitKey(1) & 0xff==ord('q'):   # used for closing the loop 
        break
cap.release()        # deacitvating the cam which we used now
cv2.destroyAllWindows()   # closing the all related windows which are opened

### emotion detection

In [9]:
import cv2
import numpy as np
from deepface import DeepFace

faceCascade=cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
font=cv2.FONT_HERSHEY_SIMPLEX
def exp():
    ret,frame= cap.read()
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces= faceCascade.detectMultiScale(gray,1.1,4)
    if(len(faces)!=0):
        res=DeepFace.analyze(frame, actions=['emotion'])
        for (x,y,w,h) in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),1) 
            cv2.putText(frame, res['dominant_emotion'],(x-10,y-10),font,3,(0,0,255),cv2.LINE_4)
        print(res['dominant_emotion'])
    cv2.imshow('testing emotions',frame)  


cap=cv2.VideoCapture(0)
while True:
    try:
        exp()
    except:
        print("no face found")
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

### PROPOSED SYSTEM

## Main Code

In [10]:
import cv2
import numpy as np
import face_recognition
import os
from deepface import DeepFace
import time

In [11]:
path = r'C:\Users\JOKER\Documents\Face_exp\CSE-1A PHOTOS'
images = []
classNames = []
myList = os.listdir(path)
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])

encodeListKnown=np.load('encodeface.npy')

In [12]:
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    
    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)
    try:
        for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
            matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
            faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
            matchIndex = np.argmin(faceDis)
    
            if matches[matchIndex]:
                res=DeepFace.analyze(img, actions=['emotion'])
                name = classNames[matchIndex].upper()
                print(name,res['dominant_emotion'])
                y1, x2, y2, x1 = faceLoc
                y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                cv2.putText(img, name+' '+res['dominant_emotion'], (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                
    except:
        print('no face found')
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) & 0xff==ord('q'):
        break
    #time.sleep(1.200)
cap.release()
cv2.destroyAllWindows()
print('success')


1/1 [==============================] - 0s 59ms/step
S170001 neutral
1/1 [==============================] - 0s 81ms/step
S171100 neutral
1/1 [==============================] - 0s 49ms/step
S171100 neutral
1/1 [==============================] - 0s 61ms/step
S171100 neutral
1/1 [==============================] - 0s 51ms/step
S171100 neutral
1/1 [==============================] - 0s 49ms/step
S171100 neutral
1/1 [==============================] - 0s 71ms/step
S171100 neutral
no face found
success


In [4]:
print(record_data)

{'S171100': {'neutral': 1}}


In [25]:
a={'name':{'ang':6,'fun':7}}

In [26]:
z='name'
a[z]

{'ang': 6, 'fun': 7}

In [32]:
a[z]['ang']+=1
a[z]

{'ang': 8, 'fun': 7}

In [34]:
a[z].update({"hu":8})

In [36]:
a.update({"age":{'hi':1}})

In [40]:
a={'name': {'ang': 8, 'fun': 7, 'hu': 8}, 'age': {'hi': 1}}

In [44]:
n='name'
if n in a:
    if 'ang' in a[n]:
        print(1)

In [27]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList
encodeListKnown = findEncodings(images)

In [29]:
np.save('encodeface.npy',encodeListKnown)

In [30]:
encodeListKnow=np.load('encodeface.npy',allow_pickle=True)
print(encodeListKnow)

[[-0.16548252  0.15445173  0.11325142 ...  0.02390457 -0.01240396
  -0.01361537]
 [-0.17030059 -0.04403138  0.02492769 ... -0.09167479  0.01475794
   0.02743253]
 [-0.11996824  0.05773755  0.05165917 ... -0.11069394  0.16518173
   0.13106677]
 ...
 [-0.17789404  0.03328991  0.06130794 ... -0.09026919  0.06856556
  -0.01468044]
 [-0.12261506  0.00243901  0.0147264  ... -0.04825155  0.08865346
   0.03763093]
 [-0.19185492  0.00264468  0.04828811 ... -0.07156402  0.07105755
   0.03360137]]


In [37]:
'''
faceCascade=cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
font=cv2.FONT_HERSHEY_SIMPLEX
def exp():
    ret,frame= cap.read()
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces= faceCascade.detectMultiScale(gray,1.1,4)
    facesCurFrame = face_recognition.face_locations(frame)
    encodesCurFrame = face_recognition.face_encodings(frame, facesCurFrame)
    if(len(faces)!=0):
        for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
            matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
            faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
            matchIndex = np.argmin(faceDis)

            if matches[matchIndex]:
                name = classNames[matchIndex].upper()
                res=DeepFace.analyze(frame, actions=['emotion'])
                desplay=[name,res['dominant_emotion']]
                print(desplay)
                y1, x2, y2, x1 = faceLoc
                y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                cv2.putText(img, desplay[0]+desplay[1], (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
        cv2.imshow('Webcam', img)
        for (x,y,w,h) in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),1) 
            cv2.putText(frame, res['dominant_emotion'],(x-10,y-10),font,3,(0,0,255),cv2.LINE_4)
        print(res['dominant_emotion'])
    cv2.imshow('testing emotions',frame)
                 
cap=cv2.VideoCapture(0)
while True:
    try:
        exp()
    except:
        print("no face found")
    if cv2.waitKey(0) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()
'''   

In [15]:
cap.release()